<h1>Обучаем модель для классификации</h1>

In [1]:
# -*- coding: utf-8 -*-
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from lxml import objectify
from pandas import DataFrame
from datetime import datetime, date, time
from dateutil.parser import parse
import pandas as pd
import numpy as np
import sklearn as sk
import time
import sys
from sklearn import metrics
import re

<h2>Источник данных</h2>

In [ ]:
%%time
#В папке TMdata лежит наш XML файл который будем читать. В итоге в Root будут распарсенные XML элементы
path = 'TMdata/xmlWikiOrderByDate.xml'
parsed = objectify.parse(open(path))
root = parsed.getroot()

In [2]:
stop_words = ['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', \
             'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', \
             'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', \
             'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', \
             'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', \
             'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', \
             'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', \
             'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', \
             'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', \
             'много', 'разве', 'три', 'эту', 'моя', 'впрочем', 'хорошо', 'свою', 'этой', 'перед', 'иногда', 'лучше', 'чуть', \
             'том', 'нельзя', 'такой', 'им', 'более', 'всегда', 'конечно', 'всю', 'между']

<h2>DataFrame</h2>

In [ ]:
%%time
#Последняя рабочая долгая версия 
dftime = pd.DataFrame (columns=('id', 'data', 'category', 'title', 'content'))
skip_fieldst2 = ['text', 'userid', 'title', 'data', 'autor', 'id', 'content']
for eltime in root.page:
    for childt in eltime.getchildren():
        if childt.tag in skip_fieldst2:
            continue
        dout=re.sub(r'(\d+)-(\d+)-(\d+)', r'01-\2-\3', eltime.data.pyval)
        dd=datetime.strptime(dout, '%d-%m-%Y')
        #dd=dd.resample("Y")
        rowt = dict(zip(['id', 'data', 'category', 'title', 'content'], 
                        [eltime.id.pyval, dd, childt.pyval, eltime.title.pyval, eltime.content.pyval]))
        row_st = pd.Series(rowt)
        row_st.name=eltime.id.pyval
        dftime = dftime.append(row_st)

In [ ]:
#Сохраняем датафрейм, чтобы постоянно не считать
dftime.to_pickle('TMdata/dftime.pkl')

In [3]:
#Загрузка из датафрейма
dftime = pd.read_pickle('TMdata/dftime.pkl')

In [4]:
dftime.head()

,id,data,category,title,content
7376,7376,2005-02-01,Барнаул,Умер Владимир Мефодьевич Башунов,\n Сегодня утром из Барнаула пришло печал...
7376,7376,2005-02-01,Алтайский край,Умер Владимир Мефодьевич Башунов,\n Сегодня утром из Барнаула пришло печал...
7376,7376,2005-02-01,Некрологи,Умер Владимир Мефодьевич Башунов,\n Сегодня утром из Барнаула пришло печал...
7376,7376,2005-02-01,Писатели,Умер Владимир Мефодьевич Башунов,\n Сегодня утром из Барнаула пришло печал...
7376,7376,2005-02-01,Россия,Умер Владимир Мефодьевич Башунов,\n Сегодня утром из Барнаула пришло печал...


<h2>
Выбираем категории где больше 50 документов</h2>

In [5]:
dftimeR=dftime.groupby(['category']).filter(lambda x: x['category'].value_counts() > 50)
dftimeR.head()


,id,data,category,title,content
7376,7376,2005-02-01,Некрологи,Умер Владимир Мефодьевич Башунов,\n Сегодня утром из Барнаула пришло печал...
7376,7376,2005-02-01,Россия,Умер Владимир Мефодьевич Башунов,\n Сегодня утром из Барнаула пришло печал...
1349,1349,2005-11-01,Россия,Россия удивлена заявлениями Премьер-министра У...,\n Украина\n В Москве обратили внима...
1349,1349,2005-11-01,Украина,Россия удивлена заявлениями Премьер-министра У...,\n Украина\n В Москве обратили внима...
1349,1349,2005-11-01,Политика,Россия удивлена заявлениями Премьер-министра У...,\n Украина\n В Москве обратили внима...


In [6]:
%%time
dfTimeUniqCatR=pd.unique(dftimeR.category.ravel())
print len(dfTimeUniqCatR)

230
Wall time: 4 ms


<h2>DataFrame 2</h2>

In [ ]:
%%time
df4 = pd.DataFrame(columns=('id','content', 'data', 'category'))
#i = 0
dfList = dftimeMinusCat['category'].tolist()
skip_fields4 = ['category']
for el4 in root.page:
    el_cat=[]
    for child4 in el4.getchildren():
        if child4.tag in skip_fields4:
            if child4.pyval not in dfList:
                el_cat.append(child4.pyval)
    if not el_cat:
        #print "List Empty"
        continue
    dout=re.sub(r'(\d+)-(\d+)-(\d+)', r'01-\2-\3', el4.data.pyval)
    dd=datetime.strptime(dout, '%d-%m-%Y')
    el_cat_string = ", ".join(el_cat)
    content1 = el4.title.pyval + el4.content.pyval
    row4 = dict(zip(['id', 'content', 'data', 'category'], [el4.id.pyval, content1, dd, el_cat_string]))
    row_s4 = pd.Series(row4)
    row_s4.name = el4.id.pyval
    df4 = df4.append(row_s4)
    #i = i + 1

In [ ]:
#Сохраняем датафрейм, чтобы постоянно не считать
df4.to_pickle('TMdata/df4.pkl')

In [7]:
#Загрузка из датафрейма
df4 = pd.read_pickle('TMdata/df4.pkl')

In [8]:
df4.head()

,id,content,data,category
7376,7376,Умер Владимир Мефодьевич Башунов\n Сегодн...,2005-02-01,"Некрологи, Россия"
1349,1349,Россия удивлена заявлениями Премьер-министра У...,2005-11-01,"Россия, Украина, Политика"
1353,1353,Впервые зарегистрирован свет самых древних зве...,2005-11-01,"Астрономия, Опубликовано"
1359,1359,День народного единства\n В России первый...,2005-11-01,"Россия, Праздники"
1361,1361,Беспорядки в Париже\n Франция\n В Па...,2005-11-01,"Франция, Гражданские беспорядки"


<h2>Создаем выборку данных для обучения модели и тестовых данных</h2>

In [9]:
df4.count()

id          6428
content     6428
data        6428
category    6428
dtype: int64

In [10]:
mask = (df4['data'] >= '2005-01-01') & (df4['data'] < '2012-01-01')
df4_5500=df4.loc[mask]
print (df4_5500.count())

id          2430
content     2430
data        2430
category    2430
dtype: int64


In [11]:
mask = (df4['data'] >= '2012-01-01') & (df4['data'] < '2013-01-01')
df4_5500_new=df4.loc[mask]
print (df4_5500_new.count())

id          1129
content     1129
data        1129
category    1129
dtype: int64


<h2>Создаем данные для обучения</h2>

In [12]:
category = df4_5500['category'].tolist()
content = df4_5500['content'].tolist()

In [13]:
%%time
category2=[]
for line in category:
    lines = line.strip().split(', ')
    lis=""
    for li in lines:
        cat = "_".join(li.split(' '))
        lis += cat + " "
    category2.append(lis)
#print ("\n".join(category2))

Wall time: 11 ms


In [14]:
vectorizerContent = CountVectorizer (min_df=1, stop_words=stop_words)
vectorizerCategory = CountVectorizer (min_df=1)
#vectorizerDate = CountVectorizer (min_df=1)


In [15]:
%%time
#X-pwd
pwd = vectorizerContent.fit_transform(content)
#Xcat - pdt
pdt = vectorizerCategory.fit_transform(category2)
#Xdat = vectorizerDate.fit_transform(dataT)

Wall time: 1.85 s


In [16]:
#xx-pwdT
pwdT = pwd.toarray().transpose()
#xxcat-pdtT
pdtT = pdt.toarray().transpose()

In [17]:
%%time
selected_feature_names_Cont = np.asarray(vectorizerContent.get_feature_names())
selected_feature_names_Cat = np.asarray(vectorizerCategory.get_feature_names())

Wall time: 134 ms


In [18]:
print len(selected_feature_names_Cat)
print len(selected_feature_names_Cont)

213
83849


In [ ]:
%%time
#Pwt - матрица тема документ
# Pwt = SUM(Ptd * Pdw)
i = 0
Pwt = 0
for xC in pdt:
    Pwt += np.array(xC.toarray())*np.array(pwd[i].toarray().transpose())
    #print (i)
    i += 1
#print (Pwt)

In [ ]:
#Сохраняем датафрейм, чтобы постоянно не считать
#Pwt.to_pickle('TMdata/Pwt.pkl')
np.save('TMdata/Pwt1.npy', Pwt)

In [19]:
Pwt=np.load('TMdata/Pwt1.npy')

In [20]:
len(Pwt.T[2])

83849

In [ ]:
%%time
i1 = 0
Phi = np.arange(Pwt.size, dtype=float).reshape(Pwt.sum(axis=1).size, Pwt[0].size)
for el4 in Pwt.sum(axis=1):
    i2 = 0
    for el1 in Pwt[i1]:
        Phi[i1][i2]=float(el1)/Pwt[i1].sum()
        i2 += 1
    i1 += 1
print (Phi)

In [ ]:
Phi_T=Phi.copy()
Phi_T=Phi_T.transpose()
len(Phi_T[1])


In [ ]:
df_Cat_Word = pd.DataFrame(columns=('word','probability', 'count'))

In [ ]:
%%time
i3=0
#n=100
for k in Phi_T[106]:
    #print (u"Слово %s вероятность %f" % (selected_feature_names_Cont[i3], k))
    row5 = dict(zip(['word', 'probability', 'count'], [selected_feature_names_Cont[i3], k, Pwt[i3][146]]))
    row_s5 = pd.Series(row5)
    row_s5.name = i3
    df_Cat_Word = df_Cat_Word.append(row_s5)
    i3+=1
    if i3%20000==0:
        print i3
    #if i3>n:
    #    break

In [ ]:
df_Cat_Word.sort(['probability', 'count'], ascending=[False, False] )

In [ ]:
#количество документов в коллекции
countDoc = len(content)
print (countDoc)

In [ ]:
print ("Уникальных слов в обучении = %s" % len(Pwt.sum(axis=1)))
print ("Количество категорий = %s" % len(Pwt.sum(axis=0)))
print ("Слов в коллекции = %s" % Pwt.sum())
print ("Количество документов = %s" % df4_5500['id'].count())

<h2>Мультиклассовая классификация</h2>

In [21]:
from __future__ import division

In [ ]:
#Количество категорий которые выводим
maxCat = 50
#Количество документов из тестовой выборки
bb=5

In [ ]:
%%time
a = iter(list(range(bb)))
for i in a:
    numDStart=1050+i
    numDEnd=1050+i+1
    newD = vectorizerContent.transform(df4[numDStart:numDEnd]['category'].tolist()).toarray()
    #print (newD)
    j=0
    newDT=newD.transpose()
    predictMatrix = []
    for inNew in np.nditer(newD):
        if inNew > 0:
            #print (inNew)
            wordInDoc = float(pwdT[j].sum())
            k = 0
            for inCat in Pwt[j]:
                if inCat > 0:
                    wordInCat = float(pdtT[k].sum())
                    el_data= {}
                    el_data[selected_feature_names_Cat[k]] = Phi[j][k]*(1-wordInCat/countDoc)*(1-wordInDoc/countDoc)*newDT[j]/newD.sum()
                    predictMatrix.append(dict(el_data))
                k += 1
        j += 1
    dfM = DataFrame(predictMatrix)
    del predictMatrix


In [ ]:
%%time
true1=0
false1=0
sum1=0
true2=[]
#test_doc_=random.randint(6000, 7000)

a = iter(list(range(bb)))
for i in a:
    numDStart=1050+i
    numDEnd=1050+i+1
    newD = vectorizerContent.transform(df4[numDStart:numDEnd]['category'].tolist()).toarray()
    #print (newD)
    j=0
    newDT=newD.transpose()
    predictMatrix = []
    for inNew in np.nditer(newD):
        if inNew > 0:
            #print (inNew)
            wordInDoc = float(pwdT[j].sum())
            k = 0
            for inCat in Pwt[j]:
                if inCat > 0:
                    wordInCat = float(pdtT[k].sum())
                    el_data= {}
                    el_data[selected_feature_names_Cat[k]] = Phi[j][k]*(1-wordInCat/countDoc)*(1-wordInDoc/countDoc)*newDT[j]/newD.sum()
                    predictMatrix.append(dict(el_data))
                k += 1
        j += 1
    dfM = DataFrame(predictMatrix)
    del predictMatrix
    #print (dfM.head())
    
    list_test_cat = df4[numDStart:numDEnd]['category'].tolist()
    category2_test_=[]
    for line in list_test_cat:
        lines = line.strip().split(', ')
        lis=""
        for li in lines:
            cat = "_".join(li.lower().split(' '))
            category2_test_.append(cat)
    dfSumM=dfM.sum()
    
    n = 1
    m=0
    dfSumM.sort_values(ascending=False, kind='quicksort', na_position='last', inplace=True)
    for el in dfSumM.T.iteritems():
        sum1+=1
        if n < maxCat:
            if el[0] in category2_test_:
                if n==1:
                    true1+=1
                #print ("%s) BINGO %s = %f" % (n, el[0], el[1]))
                m += 1
            else:
                false1+=1
            
            n += 1
    #print (", ".join(category2_test_))
    true2.append(float((m)/len(category2_test_)))
    #print ("%s из %s" % (m, len(category2_test_)))
        
print ("True %s in %s" % (true1, bb))
print ("True %f in maxCat %s" % (sum(true2)/float(len(true2)), maxCat))
print ("False %f " % (float(false1)/sum1))

In [ ]:
%%time

newWord = vectorizerContent.transform([u"выборы президента России"]).toarray()
#print (newWord)
predictMatrixW = []
j=0
for inNew in np.nditer(newWord):
    if inNew > 0:
        wordInDoc = float(pwdT[j].sum())
        k = 0
        for inCat in Pwt[j]:
            if inCat > 0:
                wordInCat = float(pdtT[k].sum())
                el_data= {}
                el_data[selected_feature_names_Cat[k]] = Phi[j][k]*(1-wordInCat/countDoc)*(1-wordInDoc/countDoc)
                predictMatrixW.append(dict(el_data))
            k +=1
    j+=1
dfMW = DataFrame(predictMatrixW)

#print (dfM.sum())
dfSumMW=dfMW.sum()
i =0
dfSumMW.sort_values(ascending=False, kind='quicksort', na_position='last', inplace=True)
for el in dfSumMW.T.iteritems():
    print ("%s) %s = %f" % (i, el[0], el[1]))
    #if i < 21:
    #    print ("%s) %s = %f" % (i, el[0], el[1]))
    i += 1

In [ ]:
#Сохраняем датафрейм, чтобы постоянно не считать
dfSumMW.to_pickle('TMdata/dfSumMW.pkl')

In [ ]:
#Загрузка из датафрейма
dfSumMW = pd.read_pickle('TMdata/dfSumMW.pkl')

In [ ]:
dfV=pd.DataFrame(dfSumMW, columns=['Probability'])

In [ ]:
dfV2=dfV[0:10]
rcParams['font.family'] = 'verdana'
rcParams['legend.fontsize'] = 24
#xlabel(u"Категории")
dfV2.plot.bar(figsize=(20, 10), alpha=0.5, color='000', label=u'Кат', fontsize='24');
plt.show()

In [ ]:
dfV.head()

<h2>Регуляризация</h2>

In [ ]:
%%time
i1 = 0
Phi1 = np.arange(Pwt1.size, dtype=float).reshape(Pwt1.sum(axis=1).size, Pwt1[0].size)
for el4 in Pwt1.sum(axis=1):
    i2 = 0
    for el1 in Pwt1[i1]:
        Phi1[i1][i2]=float(el1)/Pwt1[i1].sum()
        i2 += 1
    i1 += 1
print (Phi1)

In [ ]:
%%time
true1=0
false1=0
sum1=0
true2=[]
#test_doc_=random.randint(6000, 7000)

a = iter(list(range(bb)))
for i in a:
    numDStart=5750+i
    numDEnd=5750+i+1
    newD = vectorizerContent.transform(df4[numDStart:numDEnd]['category'].tolist()).toarray()
    #print (newD)
    j=0
    newDT=newD.transpose()
    predictMatrix = []
    for inNew in np.nditer(newD):
        if inNew > 0:
            #print (inNew)
            wordInDoc = float(pwdT[j].sum())
            k = 0
            for inCat in Pwt[j]:
                if inCat > 0:
                    wordInCat = float(pdtT[k].sum())
                    el_data= {}
                    el_data[selected_feature_names_Cat[k]] = Phi1[j][k]*(1-wordInCat/countDoc)*(1-wordInDoc/countDoc)*newDT[j]/newD.sum()
                    predictMatrix.append(dict(el_data))
                k += 1
        j += 1
    dfM = DataFrame(predictMatrix)
    del predictMatrix
    #print (dfM.head())
    
    list_test_cat = df4[numDStart:numDEnd]['category'].tolist()
    category2_test_=[]
    for line in list_test_cat:
        lines = line.strip().split(', ')
        lis=""
        for li in lines:
            cat = "_".join(li.lower().split(' '))
            category2_test_.append(cat)
    dfSumM=dfM.sum()
    
    n = 1
    m=0
    dfSumM.sort_values(ascending=False, kind='quicksort', na_position='last', inplace=True)
    for el in dfSumM.T.iteritems():
        sum1+=1
        if n < maxCat:
            if el[0] in category2_test_:
                if n==1:
                    true1+=1
                #print ("%s) BINGO %s = %f" % (n, el[0], el[1]))
                m += 1
            else:
                false1+=1
            
            n += 1
    #print (", ".join(category2_test_))
    true2.append(float((m)/len(category2_test_)))
    #print ("%s из %s" % (m, len(category2_test_)))
        
print ("True %s in %s" % (true1, bb))
print ("True %f in maxCat %s" % (sum(true2)/float(len(true2)), maxCat))
print ("False %f " % (float(false1)/sum1))

In [ ]:
%%time

newWord = vectorizerContent.transform([u"выборы призедента россии"]).toarray()
#print (newWord)
newWord?
predictMatrixW = []
j=0
for inNew in np.nditer(newWord):
    if inNew > 0:
        wordInDoc = float(pwdT[j].sum())
        k = 0
        for inCat in Pwt[j]:
            if inCat > 0:
                wordInCat = float(pdtT[k].sum())
                el_data= {}
                el_data[selected_feature_names_Cat[k]] = Phi[j][k]*(1-wordInCat/countDoc)*(1-wordInDoc/countDoc)
                predictMatrixW.append(dict(el_data))
            k +=1
    j+=1
dfMW = DataFrame(predictMatrixW)

#print (dfM.sum())
dfSumMW=dfMW.sum()
i =0
dfSumMW.sort_values(ascending=False, kind='quicksort', na_position='last', inplace=True)
for el in dfSumMW.T.iteritems():
    print ("%s) %s = %f" % (i, el[0], el[1]))
    #if i < 21:
    #    print ("%s) %s = %f" % (i, el[0], el[1]))
    i += 1